# Consigna del desafío 2
- Crear sus propios vectores con Gensim
basado en lo visto en clase con otro
dataset.

- Probar términos de interés y explicar
similitudes en el espacio de embeddings.

- Intentar plantear y probar tests de
analogías. 
- Graficar los embeddings
resultantes.
- Sacar conclusiones.